## Importación de librerías para los sistemas de recomendación

In [41]:
import os
import numpy as np  
import pandas as pd

from surprise import ( 
  Dataset,
  Reader,
  accuracy, 
  SVD,
  AlgoBase,
  BaselineOnly
) 

from surprise.model_selection import (
  train_test_split
)

## Clase `DataLoader`

La clase `DataLoader` está diseñada para cargar y manipular datos relacionados para un sistema de recomendación. A continuación se detalla su funcionalidad:

- Constructor (`__init__`)
  - Inicializa la instancia con tres argumentos que representan las rutas de acceso a los archivos de datos de usuarios, items y datos de interacción (ratings). Construye las rutas completas combinándolas con la ruta actual del directorio de trabajo del sistema operativo
  - Carga automáticamente los conjuntos de datos para usuarios, items y datos de interacción (ratings) utilizando el método `load_set`
- Método `load_set`
  - Recibe como argumento `name` que indica qué conjunto de datos cargar ( 'DATA', 'USER', 'ITEM' )
  - Dependiendo del valor de `name`, carga un DataFrame correspondiente desde un archivo CSV, especificando las columnas esperadas y ajustando el separador y la codificación según sea necesario
  - Para los conjuntos de datos 'DATA' y 'USER', elimina columnas no deseadas después de la carga
  - Para el conjunto de datos 'ITEM', también elimina columnas específicas no necesarias
- Método `load_dataset`
  - Crea un objeto `Dataset` de la biblioteca `surprise` a partir del conjunto de datos de interacción cargado previamente, preparándolo para su uso en modelos de recomendación.
  - Configura el `Reader` para interpretar las escalas de calificación de los ratings.
- Métodos de consulta
  - `get_user_by_id`: Busca información de un usuario por su ID, devolviendo un diccionario con detalles del usuario.
  - `get_item_by_id`: Busca información de un ítem por su ID, devolviendo un diccionario con detalles del ítem.
  - `get_rating_by_ids`: Intenta encontrar el rating dado un ID de usuario e ID de ítem, devolviendo el rating junto con un indicador de éxito (True si se encontró, False en caso contrario).
  - `get_ratings_by_name_id`: Filtra los datos de interacción para encontrar ratings basados en un criterio específico (por ejemplo, ID de género), devolviendo un DataFrame con los ratings encontrados.

In [140]:
class DataLoader: 
  
  def __init__(self, data_path: str, item_path: str, user_path: str) -> None:
    
    current = os.getcwd () [ 0 : os.getcwd ().rfind( '\\' ) ]
    self.DATA_PATH = current + data_path
    self.ITEM_PATH = current + item_path
    self.USER_PATH = current + user_path

    self.data_set = self.load_set ( 'DATA' )
    self.item_set = self.load_set ( 'ITEM' )
    self.user_set = self.load_set ( 'USER' )

  def load_set (self, name: str ) -> pd.DataFrame:

    if name == 'DATA':
      columns = [ 'userID', 'itemID', 'rating', 'timestamp' ]
      df = pd.read_csv ( 
        self.DATA_PATH, 
        names=columns, 
        sep='\t', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'timestamp' ] )
      return df
  
    if name == 'USER':
      columns = [ 'userID', 'age', 'gender', 'occupation', 'zipCode' ]
      df = pd.read_csv ( 
        self.USER_PATH, 
        names=columns, 
        sep='|', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'zipCode' ] )
      return df
  
    if name == 'ITEM':
      columns = [ 
        'itemID', 
        'name', 
        'releaseDate', 
        'videoReleaseDate', 
        'IMDbURL', 
        'gender_unknown', 
        'gender_action', 
        'gender_adventure', 
        'gender_animation', 
        'gender_children', 
        'gender_comedy',
        'gender_crime',
        'gender_documentary',
        'gender_drama',
        'gender_fantasy',
        'gender_film_noir',
        'gender_horror',
        'gender_musical',
        'gender_mystery',
        'gender_romance',
        'gender_scifi',
        'gender_thriller',
        'gender_war',
        'gender_western',
      ]
      df = pd.read_csv ( 
        self.ITEM_PATH, 
        names=columns, 
        sep='|', 
        encoding='latin-1', 
        skipinitialspace=True 
      )
      df = df.drop ( columns= [ 'videoReleaseDate', 'IMDbURL' ] )
      return df

  def load_dataset ( self ) -> Dataset:
    reader = Reader ( rating_scale= ( 1,5 ) )
    data = Dataset.load_from_df ( self.data_set [ [ 'userID', 'itemID', 'rating' ] ], reader )
    return data

  def get_user_by_id ( self, id: int ):
    info = self.user_set.loc [ self.user_set[ 'userID' ] == id ]
    return info[ [ 'userID', 'age', 'gender', 'occupation' ] ].iloc[0].to_dict()

  def get_item_by_id ( self, id: int ):
    info = self.item_set.loc [ self.item_set[ 'itemID' ] == id ]
    return info[ [ 
      'itemID', 
      'name', 
      'releaseDate', 
      'gender_unknown', 
      'gender_action', 
      'gender_adventure', 
      'gender_animation', 
      'gender_children', 
      'gender_comedy',
      'gender_crime',
      'gender_documentary',
      'gender_drama',
      'gender_fantasy',
      'gender_film_noir',
      'gender_horror',
      'gender_musical',
      'gender_mystery',
      'gender_romance',
      'gender_scifi',
      'gender_thriller',
      'gender_war',
      'gender_western', ] ].iloc[0].to_dict()

  def get_rating_by_ids ( self, user_id: int, item_id: int ):
    try:
      rating = self.data_set.loc [ self.data_set[ 'userID' ] == user_id ].loc [ self.data_set[ 'itemID' ] == item_id ]
      return ( rating.iloc[0]['rating'], True )
    except:
      # Failed to retrieve the rating
      return ( -1, False )

  def get_ratings_by_name_id ( self, column_name: str, id: int ):
    filtered_data = self.data_set.loc [ self.data_set[ column_name ] == id ]
    return filtered_data [ [ 'userID', 'itemID', 'rating' ] ]


Las direcciones donde tienen los datos para el sistema de recomendación son:

In [80]:
DATA_PATH = '\\dataset\\data.csv'
ITEM_PATH = '\\dataset\\item.csv'
USER_PATH = '\\dataset\\user.csv'

In [141]:
loader = DataLoader( 
  data_path=DATA_PATH,
  item_path=ITEM_PATH,
  user_path=USER_PATH 
)

In [142]:
loader.data_set
#loader.item_set
#loader.user_set

,userID,itemID,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [143]:
ratings_by_user_id = loader.get_ratings_by_name_id ( column_name='userID', id=196 )

results = f"""
========================================

Results Dataframe sort values by item id
{ ratings_by_user_id.sort_values ( by='itemID', ascending=True ) }

========================================

Results Dataframe sort values by rating
{ ratings_by_user_id.sort_values ( by='rating', ascending=False ) }

========================================

Shape: { ratings_by_user_id.shape }

"""
print ( results )

rating_by_ids = loader.get_rating_by_ids ( item_id=242, user_id=196 )
print ( f'Rating: { rating_by_ids }' )

info_item = loader.get_item_by_id ( id=242 )
print ( info_item )

info_user = loader.get_user_by_id ( id=196 )
print ( info_user )




Results Dataframe sort values by item id
       userID  itemID  rating
17102     196       8       5
56628     196      13       2
10981     196      25       4
22271     196      66       3
2374      196      67       5
21605     196      70       3
14606     196      94       3
23189     196     108       4
87863     196     110       1
10017     196     111       4
33536     196     116       3
52726     196     153       5
59607     196     173       2
24030     196     202       3
7517      196     238       4
0         196     242       3
1812      196     251       3
22773     196     257       2
78787     196     269       3
36281     196     285       5
13733     196     286       5
32721     196     287       3
6910      196     306       4
25726     196     340       3
1133      196     381       4
35197     196     382       4
940       196     393       4
50147     196     411       4
17830     196     428       4
10254     196     580       2
1896      196     655      

## Clase `Exploratory_Data_Analysis`

El Análisis Exploratorio de Datos permite determinar la mejor manera de manipular lsa fuentes de datos para obtener las respuesta que necesita, permitiendo descubrir patrones, detectar anomalías, probar una hipótesis o verificar suposiciones. 

El objetivo de tener una clase para el Análisis Exploratorio de Datos es ayudar a analizar los datos antes de hacer las predicciones. Permitiendo en algunos casos identificar errores obvios, así como comprender mejor los patrones dentro de los datos e incluso encontrar relaciones interesantes entre las variables. 

Herramientas que pudieran usarse para el Análisis Exploratorio de Datos: 
- Técnicas de clustering y reducción de dimensiones
- Visualización univariante de cada campo del conjunto de datos sin procesar, con estadísticas de resumen
- Visualización bivariante y estadísticas de resumen que le permiten evaluar la relación entre cada variable del conjunto de datos y la variable de destino que está viendo
- Visualización multivariantes, para mapear y comprender las interacciones entre los diferentes campos de datos 
- Métodos predictivos como la regresión lineal permite la predicción de resultados

In [85]:
class Exploratory_Data_Analysis: 
  pass



In [46]:
class Metrics: 
  def __init__(self, predictions) -> None:
    self.predictions = predictions
    self.metrics = { }

  def compute_metrics ( self, *args ) -> dict :
    if 'MAE' in args: self.MAE ( )
    if 'RMSE' in args: self.RMSE ( )
    return self.metrics

  def MAE  ( self ):
    self.metrics [ 'MAE' ] = accuracy.mae ( self.predictions ) 

  def RMSE ( self ): 
    self.metrics [ 'RMSE' ] = accuracy.rmse ( self.predictions ) 

In [47]:
class DataGenerator: 
  def __init__(self, data, percentage = 0.25) -> None:
    """Data Generator

    Build a 75/25 train/test split for measuring accuracy

    Args:
        percentage (float, optional): _description_. Defaults to 0.25.
    """
    self.trainset, self.testset = train_test_split ( data, test_size=percentage, random_state=1 )

  def get_trainset( self ):
    return self.trainset
  
  def get_testset( self ):
    return self.testset

In [48]:
class Model: 
  def __init__(self, model, name) -> None:
    self.model = model
    self.name = name
  
  def __str__(self) -> str:
    return f'Model: { self.name }'
  
  def evaluate ( self, data: DataGenerator ): 
    predictions = self.model.fit ( data.get_trainset() ).test ( data.get_testset() )
    metrics = Metrics ( predictions ).compute_metrics( 'MAE', 'RMSE' )
    return metrics

In [49]:
class Factory:
  def __init__(self, dataset) -> None:
    self.dataset = DataGenerator ( dataset )
    self.models: list[ Model ] = [ ]
  
  def add_model ( self, model: Model ):
    self.models.append ( model )
  
  def evaluate ( self ):
    results = { }
    for model in self.models:
      print ( f'Evaluating { model.name }' )
      results [ model.name ] = model.evaluate( self.dataset )

  def clean_models ( self ):
    self.models = [] 


In [66]:
model_svd = Model ( model=SVD(), name='SVD' )
factory = Factory( loader.load_dataset( ) )

factory.add_model( model_svd )
factory.evaluate()

Evaluating SVD
MAE:  0.7402
RMSE: 0.9411


In [67]:
model_baseline = Model ( model=BaselineOnly(), name='Baseline Only' )
factory = Factory( loader.load_dataset( ) )

factory.add_model( model_baseline )
factory.evaluate()

Evaluating Baseline Only
Estimating biases using als...
MAE:  0.7501
RMSE: 0.9485


In [72]:
class HybridModel ( AlgoBase ):

  def __init__ (self, models, weights, **kwargs):
    super().__init__(**kwargs) 
    self.models: list[ Model ] = models
    self.weights = weights
  
  def fit (self, trainset):
    AlgoBase.fit ( self, trainset )
    for model in self.models:
      model.model.fit ( trainset )
    return self
  
  def estimate ( self, user_id, item_id ):
    scores = 0 
    weight = 0
    for i in range ( len( self.models ) ):
      scores += self.models[i].model.predict ( user_id, item_id ).est * self.weights[i]
      weight += self.weights[i]

    return scores/weight

In [71]:
models = [ 
  model_svd,
  model_baseline
]
weights = [ 
  0.5,
  0.5
]

hybrid = HybridModel ( models, weights )

factory = Factory ( loader.load_dataset() )

model = Model ( hybrid, 'Hybrid: SVD - Baseline' )
factory.add_model ( model )

factory.evaluate()

Evaluating Hybrid: SVD - Baseline
Estimating biases using als...
MAE:  0.9949
RMSE: 1.2280


TODO: 
- Completar la clase de EDA
- Mejorar los sistemas de recomendacion hibridos